In [15]:
import pandas as pd
import pickle
import json
import tweepy
import time
import numpy as np
import re
import networkx as nx
import wget
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import re
from nltk.corpus import stopwords
import csv

##  read saved pickle
### This notebook contains steps for data preprocessing, everything is pickled for easier access. 
####  data cleaning, classify location, tweets tokenized, . refer to next jupyter notebook for sentiment analysis code.
#### tweetfile2 contains individual tokenized tweets 
#### locationtokenized conbines tweets from the same country into one corpus.

In [120]:
with open('tweetfile1.p', 'rb') as fp:
    pd1 = pickle.load(fp)
with open('tweetfile2.p', 'rb') as fp:
    pd2 = pickle.load(fp)
with open('locationtokenized.p', 'rb') as fp:
    pd3 = pickle.load(fp)
with open('tweetstring.p', 'rb') as fp:
    pd4 = pickle.load(fp)
with open('locationstring.p', 'rb') as fp:
    pd5 = pickle.load(fp)

In [121]:
pd2.head()

,tweet,location,created
0,"[well, looking, like, amy, coney, barrett, ann...",US,02-Oct-2020
1,"[wapo, deleted, tweet, sent, hour, trump, anno...",US,02-Oct-2020
8,"[tree4justice, real, thing, nurse, friends, se...",US,02-Oct-2020
9,"[covid19, officially, denounces, white, suprem...",US,02-Oct-2020
11,"[statement, president, trumps, covid19, diagno...",CANADA,02-Oct-2020


In [122]:
pd3.head()

,tweet
location,
ARGENTINA,"[statement, trumps, covid19, positive, diagnos..."
AUSTRALIA,"[coronavirus, ultimate, trap, media, loselose,..."
BELGIUM,"[kary, mullis, invented, pcr, nobel, prize, ch..."
BRAZIL,"[alarming, data, shows, third, wave, covid19, ..."
CANADA,"[statement, president, trumps, covid19, diagno..."


## read in all covidtweets json files and store them into a single dataframe

In [3]:
filename='covidtweets2020-10-'

In [4]:
finalpd = pd.DataFrame({'tweet' : [],'location':[],'created':[]})

In [5]:

for i in range(1,32):
    try:
        if i<10:
            filename1=filename+'0'+str(i)+'.json'
            with open(filename1, 'r', encoding='utf-8') as json_file:
                languages_dict = json.load(json_file)
                data2=pd.DataFrame(languages_dict)  
                finalpd=finalpd.append(data2,ignore_index=True)
                print(filename1)
        else:
            filename1=filename+str(i)+'.json'
            with open(filename1, 'r', encoding='utf-8') as json_file:
                languages_dict = json.load(json_file)
                data2=pd.DataFrame(languages_dict)  
                finalpd=finalpd.append(data2,ignore_index=True)
                print(filename1)
    except Exception as e:
        print(e)
        continue
    

[Errno 2] No such file or directory: 'covidtweets2020-10-01.json'
covidtweets2020-10-02.json
covidtweets2020-10-03.json
covidtweets2020-10-04.json
covidtweets2020-10-05.json
covidtweets2020-10-06.json
covidtweets2020-10-07.json
covidtweets2020-10-08.json
covidtweets2020-10-09.json
covidtweets2020-10-10.json
covidtweets2020-10-11.json
covidtweets2020-10-12.json
covidtweets2020-10-13.json
[Errno 2] No such file or directory: 'covidtweets2020-10-14.json'
covidtweets2020-10-15.json
covidtweets2020-10-16.json
covidtweets2020-10-17.json
covidtweets2020-10-18.json
covidtweets2020-10-19.json
covidtweets2020-10-20.json
covidtweets2020-10-21.json
covidtweets2020-10-22.json
covidtweets2020-10-23.json
[Errno 2] No such file or directory: 'covidtweets2020-10-24.json'
covidtweets2020-10-25.json
covidtweets2020-10-26.json
[Errno 2] No such file or directory: 'covidtweets2020-10-27.json'
covidtweets2020-10-28.json
covidtweets2020-10-29.json
covidtweets2020-10-30.json
covidtweets2020-10-31.json


In [6]:
with open('covidtweets2020-11-01.json', 'r', encoding='utf-8') as json_file:
                languages_dict = json.load(json_file)
                data2=pd.DataFrame(languages_dict)  
                finalpd=finalpd.append(data2,ignore_index=True)
                print(filename1)

covidtweets2020-10-31.json


In [7]:
finalpd=finalpd.drop(columns=['following','followers','retweetcount'])

### remove rows without location 

In [8]:
finalpd['location'].replace('', np.nan, inplace=True)

### Data cleaning 

In [9]:
def standardize_text(text):
    text = text.lower()
    text = re.sub("[.,!?®:@'&$’\"\-()]", "", text)
    text = re.sub(r"http.+$", " ", text)
    text = text.strip()
    text = re.sub("[#_]", "", text)
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', text) # remove URLs
    text = re.sub('@[^\s]+', 'AT_USER', text) # remove usernames
    text = re.sub(r'#([^\s]+)', r'\1', text) # remove the # in #hashtag
    #tweet = word_tokenize(tweet)
    return text

In [10]:
def toeknize_stem_token(text):
    tokens = word_tokenize(text)
    lemma = nltk.wordnet.WordNetLemmatizer()
    lemm_tokens = []

    for token in tokens:
        lemm_tokens.append(lemma.lemmatize(token))

    return lemm_tokens


In [11]:
def remove_stopwords(tokens):
    sw = stopwords.words()
    sw.append('rt')
    # I am using list comprehension here
    word_token = [t for t in tokens if t not in sw]
    #remove username
    word_token=word_token[1:]
    return word_token

In [12]:
def listToString(s):  
    str1 = " " 
    return (str1.join(s)) 

In [13]:
def cleantweets(pandaf):
    
    data2=pandaf
    
    data2['tweet'] = data2['tweet'].apply(standardize_text)

    data2['tweet'] = data2['tweet'].apply(toeknize_stem_token)

    data2['tweet'] = data2['tweet'].apply(remove_stopwords)
    
    #data2['tweet'] = data2['tweet'].apply(listToString)
    
    data2 = data2[data2.location != '']
    
    return data2

In [16]:
finalclean=cleantweets(finalpd)

In [18]:
with open('tweetfile1.p', 'wb') as fp:
    pickle.dump(finalclean, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [19]:
final1=finalclean.copy()

In [20]:
finalclean1=finalclean.copy()
finalclean1['tweet'] = finalclean1['tweet'].apply(listToString)

In [21]:
final1

,tweet,location,created
0,"[well, looking, like, amy, coney, barrett, ann...","New York, New York",02-Oct-2020
1,"[wapo, deleted, tweet, wa, sent, hour, trump, ...","Troy, AL",02-Oct-2020
2,"[best, covid19, experience, nobody, covid, wa,...",she/they,02-Oct-2020
3,"[trump, supporter, demanding, sympathize, trum...",NaN,02-Oct-2020
4,"[last, day, freakin, badass, rebellious, dream...",NaN,02-Oct-2020
...,...,...,...
207995,"[ideal, world, kid, school, remain, open, pres...","Barnsley, England",01-Nov-2020
207996,"[would, indeed, ironic, donald, trump, lost, p...",a diner in the 12th dimension,01-Nov-2020
207997,"[lil, halloween, party, worth, getting, sick, ...",Namjoon Nation,01-Nov-2020
207998,"[latest, covid19, daily, case, increase, total...",NaN,01-Nov-2020


### location cleaning 

In [22]:
final1=final1.dropna()

#### keep the value after comma, remove space 

In [23]:
def cleanlocation(text):
    
    text=text.upper()
    
    textlist=text.split(',')
    location=textlist[-1]
     
    location = location.strip()
    return location

In [24]:
final1['location'] = final1['location'].apply(cleanlocation)


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [25]:
final1

,tweet,location,created
0,"[well, looking, like, amy, coney, barrett, ann...",NEW YORK,02-Oct-2020
1,"[wapo, deleted, tweet, wa, sent, hour, trump, ...",AL,02-Oct-2020
2,"[best, covid19, experience, nobody, covid, wa,...",SHE/THEY,02-Oct-2020
8,"[tree4justice, real, thing, nurse, friend, see...",CA,02-Oct-2020
9,"[covid19, officially, denounces, white, suprem...",KENTUCKY,02-Oct-2020
...,...,...,...
207990,"[america, lead, 1, covid19, case, 1, medical, ...",TX,01-Nov-2020
207994,"[zxretroxx, sn123101, jannermaid1, sure, famil...",UK,01-Nov-2020
207995,"[ideal, world, kid, school, remain, open, pres...",ENGLAND,01-Nov-2020
207996,"[would, indeed, ironic, donald, trump, lost, p...",A DINER IN THE 12TH DIMENSION,01-Nov-2020


### get frequency of the locations

In [26]:
final1.location.mode()

0    USA
dtype: object

In [27]:
rownum=len(final1.index)
rownum

137316

In [28]:
n=0.05
validnum=int(round(rownum*n,0))
validnum

6866

### testing phase, remove location that occur less than 2

In [29]:
final1=final1.groupby("location").filter(lambda x: len(x) > 1)


#### remove location that occur less than 51

In [30]:
final1=final1.groupby("location").filter(lambda x: len(x) > 50)

#### check the occurance 

In [31]:
final1.location.value_counts()


USA              14452
CA                4913
ENGLAND           3799
UNITED STATES     3765
TX                2733
                 ...  
ND                  53
MONTANA             52
ESSEX               51
IDAHO               51
BALTIMORE           51
Name: location, Length: 218, dtype: int64

## read covid world data

In [32]:
covidworld=pd.read_csv('covid19_world-2.csv')


In [33]:
covidworld.groupby(['ObservationDate','Country_Region'])['Confirmed'].agg('sum')


ObservationDate  Country_Region    
2020-01-22       China                   547
                 Hong Kong                 0
                 Japan                     2
                 Macau                     1
                 South Korea               1
                                       ...  
2020-11-22       West Bank and Gaza    71644
                 Western Sahara           10
                 Yemen                  2099
                 Zambia                17424
                 Zimbabwe               9220
Name: Confirmed, Length: 48259, dtype: int64

In [34]:
covidworld=covidworld[(covidworld['ObservationDate'] > '2020-10-01') & (covidworld['ObservationDate'] < '2020-11-02')]




In [35]:
covidworld


,ObservationDate,Country_Region,Province_State,Confirmed,Deaths,Recovered,Active
697509,2020-10-02,Afghanistan,NaN,39290,1458.0,32842.0,4990.0
697510,2020-10-02,Albania,NaN,13965,389.0,8342.0,5234.0
697511,2020-10-02,Algeria,NaN,51847,1749.0,36385.0,13713.0
697512,2020-10-02,Andorra,NaN,2110,53.0,1540.0,517.0
697513,2020-10-02,Angola,NaN,5211,189.0,2215.0,2807.0
...,...,...,...,...,...,...,...
820166,2020-11-01,West Bank and Gaza,NaN,54060,489.0,46773.0,6798.0
820167,2020-11-01,Western Sahara,NaN,10,1.0,8.0,1.0
820168,2020-11-01,Yemen,NaN,2063,600.0,1366.0,97.0
820169,2020-11-01,Zambia,NaN,16480,349.0,15733.0,398.0


In [36]:
covidworld.groupby(['Country_Region'])['Confirmed','Deaths','Recovered','Active'].agg('sum')



/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,Confirmed,Deaths,Recovered,Active
Country_Region,,,,
Afghanistan,1248137,46303.0,1040764.0,161070.0
Albania,530480,13861.0,308539.0,208080.0
Algeria,1690213,57424.0,1181170.0,451519.0
Andorra,105842,1917.0,71161.0,32764.0
Angola,239270,7426.0,97014.0,134830.0
...,...,...,...,...
West Bank and Gaza,1458863,12621.0,1247424.0,198818.0
Western Sahara,310,31.0,248.0,31.0
Yemen,63678,18475.0,41558.0,3645.0


In [37]:
countrypopulation=pd.read_csv('world_wise.csv')


In [38]:
countrypopulation[['Country/Region','Population']]


,Country/Region,Population
0,US,3.311981e+08
1,Brazil,2.127107e+08
2,India,1.381345e+09
3,Russia,1.459409e+08
4,South Africa,5.938157e+07
...,...,...
204,Montserrat,4.992000e+03
205,Caribbean Netherlands,2.624700e+04
206,Falkland Islands,3.489000e+03
207,Vatican City,8.010000e+02


In [39]:
cname=countrypopulation['Country/Region'].tolist()
c_dict = dict(zip(countrypopulation['Country/Region'], countrypopulation['Population']))



In [40]:
cname

['US',
 'Brazil',
 'India',
 'Russia',
 'South Africa',
 'Mexico',
 'Peru',
 'Chile',
 'Colombia',
 'Spain',
 'Iran',
 'United Kingdom',
 'Saudi Arabia',
 'Pakistan',
 'Bangladesh',
 'Italy',
 'Turkey',
 'Argentina',
 'Germany',
 'France',
 'Iraq',
 'Philippines',
 'Indonesia',
 'Canada',
 'Qatar',
 'Kazakhstan',
 'Egypt',
 'Ecuador',
 'Bolivia',
 'Sweden',
 'Oman',
 'Israel',
 'Ukraine',
 'Dominican Republic',
 'Panama',
 'Belgium',
 'Kuwait',
 'Belarus',
 'UAE',
 'Romania',
 'Netherlands',
 'Singapore',
 'Guatemala',
 'Portugal',
 'Poland',
 'Nigeria',
 'Honduras',
 'Bahrain',
 'Japan',
 'Armenia',
 'Ghana',
 'Kyrgyzstan',
 'Afghanistan',
 'Switzerland',
 'Algeria',
 'Azerbaijan',
 'Morocco',
 'Uzbekistan',
 'Serbia',
 'Moldova',
 'Ireland',
 'Kenya',
 'Venezuela',
 'Nepal',
 'Austria',
 'Costa Rica',
 'Ethiopia',
 'Australia',
 'El Salvador',
 'Czechia',
 'Cameroon',
 'Ivory Coast',
 'S. Korea',
 'Denmark',
 'Palestine',
 'Bosnia and Herzegovina',
 'Bulgaria',
 'Madagascar',
 'Sudan

In [41]:
c_dict

{'US': 331198130.0,
 'Brazil': 212710692.0,
 'India': 1381344997.0,
 'Russia': 145940924.0,
 'South Africa': 59381566.0,
 'Mexico': 129066160.0,
 'Peru': 33016319.0,
 'Chile': 19132514.0,
 'Colombia': 50936262.0,
 'Spain': 46756648.0,
 'Iran': 84097623.0,
 'United Kingdom': 67922029.0,
 'Saudi Arabia': 34865919.0,
 'Pakistan': 221295851.0,
 'Bangladesh': 164851401.0,
 'Italy': 60452568.0,
 'Turkey': 84428331.0,
 'Argentina': 45236884.0,
 'Germany': 83811260.0,
 'France': 65288306.0,
 'Iraq': 40306025.0,
 'Philippines': 109722719.0,
 'Indonesia': 273808365.0,
 'Canada': 37775022.0,
 'Qatar': 2807805.0,
 'Kazakhstan': 18798667.0,
 'Egypt': 102516525.0,
 'Ecuador': 17668824.0,
 'Bolivia': 11688459.0,
 'Sweden': 10105596.0,
 'Oman': 5118446.0,
 'Israel': 9197590.0,
 'Ukraine': 43705858.0,
 'Dominican Republic': 10858648.0,
 'Panama': 4321282.0,
 'Belgium': 11594739.0,
 'Kuwait': 4276658.0,
 'Belarus': 9449001.0,
 'UAE': 9902079.0,
 'Romania': 19224023.0,
 'Netherlands': 17138756.0,
 'Singa

In [42]:
final1['location']

0              NEW YORK
1                    AL
8                    CA
9              KENTUCKY
11              TORONTO
              ...      
207982    UNITED STATES
207988              USA
207990               TX
207994               UK
207995          ENGLAND
Name: location, Length: 89646, dtype: object

In [43]:
tloc=final1.location.tolist()
tloc=set(tloc)
tloc

{'',
 'AB',
 'ABUJA',
 'AK',
 'AL',
 'ALABAMA',
 'ALASKA',
 'ALBERTA',
 'AMERICA',
 'AR',
 'ARGENTINA',
 'ARIZONA',
 'ARKANSAS',
 'ATLANTA',
 'AUSTRALIA',
 'AZ',
 'B.C.',
 'BALTIMORE',
 'BAY AREA',
 'BC',
 'BELFAST',
 'BELGIUM',
 'BOSTON',
 'BRASIL',
 'BRITISH COLUMBIA',
 'BROOKLYN',
 'CA',
 'CALIFORNIA',
 'CANADA',
 'CHICAGO',
 'CO',
 'COLOMBIA',
 'COLORADO',
 'CONNECTICUT',
 'CT',
 'D.C.',
 'DC',
 'DE',
 'DENVER',
 'DEUTSCHLAND',
 'DUBLIN',
 'EARTH',
 'EDINBURGH',
 'ENGLAND',
 'ESPAÑA',
 'ESSEX',
 'EUROPE',
 'EVERYWHERE',
 'FL',
 'FLORIDA',
 'FRANCE',
 'GA',
 'GEORGIA',
 'GERMANY',
 'GHANA',
 'GLASGOW',
 'GLOBAL',
 'HAWAII',
 'HE/HIM',
 'HELL',
 'HERE',
 'HI',
 'HONG KONG',
 'HOUSTON',
 'IA',
 'ID',
 'IDAHO',
 'IL',
 'ILLINOIS',
 'IN',
 'INDIA',
 'INDIANA',
 'INDONESIA',
 'IOWA',
 'IRELAND',
 'ITALY',
 'JAMAICA',
 'JAPAN',
 'KANSAS',
 'KENTUCKY',
 'KENYA',
 'KS',
 'KY',
 'LA',
 'LAGOS',
 'LAS VEGAS',
 'LIVERPOOL',
 'LONDON',
 'LOS ANGELES',
 'LOUISIANA',
 'MA',
 'MAINE',
 'MALAYSIA',

In [44]:
US=[]
with open('state_names.txt') as f:
    next(f)
    #lines=f.readlines()
    for line in f:
        if line!='':
            US.append(line.rstrip())

In [45]:
US.extend(('United States','U.S.A','USA','America','ATLANTA',
           'SAN FRANCISCO','D.C.','SOUTH FLORIDA','SEATTLE','SOUTHERN CALIFORNIA','CHICAGO',
          'ONTARIO','WASHINGTON DC','PHILADELPHIA','NEW YORK CITY','HOUSTON','UNITED STATES OF AMERICA',
          'DENVER','SAN FRANCISCO BAY AREA','NYC','WASHINGTON STATE','BOSTON','BALTIMORE','NEW ENGLAND',
          'BAY AREA','NORTHERN CALIFORNIA','SOCAL','BROOKLYN','MIDWEST','DC','SAN DIEGO','LAS VEGAS',
          'MINNEAPOLIS','LOS ANGELES','U.S.A.'))


In [46]:
US=[x.upper() for x in US]


In [47]:
cleanlocation=final1.copy()

In [50]:
cleanlocation['location'] = cleanlocation['location'].replace(US,'US')
cleanlocation['location'] = cleanlocation['location'].replace(UK,'UNITED KINGDOM')
cleanlocation['location'] = cleanlocation['location'].replace(CANADA,'CANADA')
cleanlocation['location'] = cleanlocation['location'].replace(NETHERLANDS,'NETHERLANDS')
cleanlocation['location'] = cleanlocation['location'].replace(AUS,'AUSTRALIA')
cleanlocation['location'] = cleanlocation['location'].replace(INDIA,'INDIA')
cleanlocation['location'] = cleanlocation['location'].replace(MEXICO,'MEXICO')
cleanlocation['location'] = cleanlocation['location'].replace(EUROPE,'EUROPE')
cleanlocation['location'] = cleanlocation['location'].replace(NA,'NORTH AMERICA')
cleanlocation['location'] = cleanlocation['location'].replace(SPAIN,'SPAIN')
cleanlocation['location'] = cleanlocation['location'].replace(NIGERIA,'NIGERIA')
cleanlocation['location'] = cleanlocation['location'].replace(BRAZIL,'BRAZIL')
cleanlocation['location'] = cleanlocation['location'].replace(GERMANY,'GERMANY')
cleanlocation['location'] = cleanlocation['location'].replace(UAE,'UAE')
cleanlocation['location'] = cleanlocation['location'].replace(IRELAND,'IRELAND')
















In [51]:
cleanlocation

,tweet,location,created
0,"[well, looking, like, amy, coney, barrett, ann...",US,02-Oct-2020
1,"[wapo, deleted, tweet, wa, sent, hour, trump, ...",US,02-Oct-2020
8,"[tree4justice, real, thing, nurse, friend, see...",US,02-Oct-2020
9,"[covid19, officially, denounces, white, suprem...",US,02-Oct-2020
11,"[statement, president, trump, covid19, diagnosis]",CANADA,02-Oct-2020
...,...,...,...
207982,"[2016, trump, became, president, 77000, vote, ...",US,01-Nov-2020
207988,"[trump, wa, infected, walter, reed, hospital, ...",US,01-Nov-2020
207990,"[america, lead, 1, covid19, case, 1, medical, ...",US,01-Nov-2020
207994,"[zxretroxx, sn123101, jannermaid1, sure, famil...",UNITED KINGDOM,01-Nov-2020


In [52]:
cleanlocation['location'] = cleanlocation['location'].replace(REMOVE,np.nan)

In [53]:
cleanlocation1=cleanlocation.dropna()

In [54]:
cleanlocation1

,tweet,location,created
0,"[well, looking, like, amy, coney, barrett, ann...",US,02-Oct-2020
1,"[wapo, deleted, tweet, wa, sent, hour, trump, ...",US,02-Oct-2020
8,"[tree4justice, real, thing, nurse, friend, see...",US,02-Oct-2020
9,"[covid19, officially, denounces, white, suprem...",US,02-Oct-2020
11,"[statement, president, trump, covid19, diagnosis]",CANADA,02-Oct-2020
...,...,...,...
207982,"[2016, trump, became, president, 77000, vote, ...",US,01-Nov-2020
207988,"[trump, wa, infected, walter, reed, hospital, ...",US,01-Nov-2020
207990,"[america, lead, 1, covid19, case, 1, medical, ...",US,01-Nov-2020
207994,"[zxretroxx, sn123101, jannermaid1, sure, famil...",UNITED KINGDOM,01-Nov-2020


In [55]:
cleanlocation1.location.value_counts()

US                61516
UNITED KINGDOM     9931
CANADA             4501
NIGERIA            2771
AUSTRALIA          1711
INDIA              1673
IRELAND             859
SOUTH AFRICA        414
GERMANY             381
FRANCE              371
NEW ZEALAND         282
SPAIN               234
MALAYSIA            220
KENYA               213
GHANA               194
PAKISTAN            182
NETHERLANDS         155
BRAZIL              148
EUROPE              139
MEXICO              123
NORTH AMERICA       121
JAPAN               121
PORTUGAL            114
PHILIPPINES         109
SINGAPORE           101
INDONESIA            95
JAMAICA              84
ARGENTINA            82
SWITZERLAND          82
HONG KONG            75
VENEZUELA            68
COLOMBIA             67
UAE                  66
ITALY                62
SWEDEN               58
BELGIUM              54
Name: location, dtype: int64

### pickle it 

In [56]:
with open('tweetfile2.p', 'wb') as fp:
    pickle.dump(cleanlocation1, fp, protocol=pickle.HIGHEST_PROTOCOL)


In [57]:
cleanlocation1

,tweet,location,created
0,"[well, looking, like, amy, coney, barrett, ann...",US,02-Oct-2020
1,"[wapo, deleted, tweet, wa, sent, hour, trump, ...",US,02-Oct-2020
8,"[tree4justice, real, thing, nurse, friend, see...",US,02-Oct-2020
9,"[covid19, officially, denounces, white, suprem...",US,02-Oct-2020
11,"[statement, president, trump, covid19, diagnosis]",CANADA,02-Oct-2020
...,...,...,...
207982,"[2016, trump, became, president, 77000, vote, ...",US,01-Nov-2020
207988,"[trump, wa, infected, walter, reed, hospital, ...",US,01-Nov-2020
207990,"[america, lead, 1, covid19, case, 1, medical, ...",US,01-Nov-2020
207994,"[zxretroxx, sn123101, jannermaid1, sure, famil...",UNITED KINGDOM,01-Nov-2020


In [58]:
stringtweet=cleanlocation1.copy()
stringtweet['tweet'] = stringtweet['tweet'].apply(listToString)

In [59]:
with open('tweetstring.p', 'wb') as fp:
    pickle.dump(stringtweet, fp, protocol=pickle.HIGHEST_PROTOCOL)

### test sample size, remove country with too few records

In [60]:
rownum=len(cleanlocation1.index)
n=0.05
validnum=int(round(rownum*n,0))
validnum

4369

In [61]:
rownum1=len(cleanlocation1.index)
n=0.01
validnum1=int(round(rownum1*n,0))
validnum1

874

In [62]:
rownum2=len(cleanlocation1.index)
n=0.005
validnum2=int(round(rownum2*n,0))
validnum2

437

In [63]:
completelocation1=cleanlocation1.groupby("location").filter(lambda x: len(x) > validnum)



In [64]:

completelocation1.location.value_counts()

US                61516
UNITED KINGDOM     9931
CANADA             4501
Name: location, dtype: int64

In [65]:
completelocation2=cleanlocation1.groupby("location").filter(lambda x: len(x) > validnum1)



In [66]:
completelocation2.location.value_counts()

US                61516
UNITED KINGDOM     9931
CANADA             4501
NIGERIA            2771
AUSTRALIA          1711
INDIA              1673
Name: location, dtype: int64

In [67]:
completelocation3=cleanlocation1.groupby("location").filter(lambda x: len(x) > validnum2)
completelocation3.location.value_counts()


US                61516
UNITED KINGDOM     9931
CANADA             4501
NIGERIA            2771
AUSTRALIA          1711
INDIA              1673
IRELAND             859
Name: location, dtype: int64

### group tweets together 

In [68]:
countrybasedpd=cleanlocation1.groupby('location').agg('sum').drop(columns=['created'])

In [69]:
countrystring=stringtweet.groupby('location').agg('sum').drop(columns=['created'])

In [70]:
with open('locationstring.p', 'wb') as fp:
    pickle.dump(countrystring, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [77]:
with open('locationtokenized.p', 'wb') as fp:
    pickle.dump(countrybasedpd, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [71]:
countrybasedpd

,tweet
location,
ARGENTINA,"[statement, trump, covid19, positive, diagnosi..."
AUSTRALIA,"[coronavirus, ultimate, trap, medium, loselose..."
BELGIUM,"[kary, mullis, invented, pcr, nobel, prize, ch..."
BRAZIL,"[alarming, data, show, third, wave, covid19, h..."
CANADA,"[statement, president, trump, covid19, diagnos..."
COLOMBIA,"[praying, swift, recovery, father, melania, im..."
EUROPE,"[back, normal, christmas, perfectly, safe, sch..."
FRANCE,"[hiiii, bweaking, pwesidenttwump, weceiving, c..."
GERMANY,"[forgive, skepticism, noticed, testing, positi..."


### check name of the country from tweets with name of country from worldwie.csv

In [72]:
tloc1=cleanlocation1.location.tolist()
tloc1=set(tloc1)
tloc1

{'ARGENTINA',
 'AUSTRALIA',
 'BELGIUM',
 'BRAZIL',
 'CANADA',
 'COLOMBIA',
 'EUROPE',
 'FRANCE',
 'GERMANY',
 'GHANA',
 'HONG KONG',
 'INDIA',
 'INDONESIA',
 'IRELAND',
 'ITALY',
 'JAMAICA',
 'JAPAN',
 'KENYA',
 'MALAYSIA',
 'MEXICO',
 'NETHERLANDS',
 'NEW ZEALAND',
 'NIGERIA',
 'NORTH AMERICA',
 'PAKISTAN',
 'PHILIPPINES',
 'PORTUGAL',
 'SINGAPORE',
 'SOUTH AFRICA',
 'SPAIN',
 'SWEDEN',
 'SWITZERLAND',
 'UAE',
 'UNITED KINGDOM',
 'US',
 'VENEZUELA'}

In [73]:
tloc=cleanlocation.location.tolist()
tloc=set(tloc)
tloc

{'ARGENTINA',
 'AUSTRALIA',
 'BELGIUM',
 'BRAZIL',
 'CANADA',
 'COLOMBIA',
 'EUROPE',
 'FRANCE',
 'GERMANY',
 'GHANA',
 'HONG KONG',
 'INDIA',
 'INDONESIA',
 'IRELAND',
 'ITALY',
 'JAMAICA',
 'JAPAN',
 'KENYA',
 'MALAYSIA',
 'MEXICO',
 'NETHERLANDS',
 'NEW ZEALAND',
 'NIGERIA',
 'NORTH AMERICA',
 'PAKISTAN',
 'PHILIPPINES',
 'PORTUGAL',
 'SINGAPORE',
 'SOUTH AFRICA',
 'SPAIN',
 'SWEDEN',
 'SWITZERLAND',
 'UAE',
 'UNITED KINGDOM',
 'US',
 'VENEZUELA',
 nan}

In [74]:
cname=[x.upper() for x in cname]

cname


['US',
 'BRAZIL',
 'INDIA',
 'RUSSIA',
 'SOUTH AFRICA',
 'MEXICO',
 'PERU',
 'CHILE',
 'COLOMBIA',
 'SPAIN',
 'IRAN',
 'UNITED KINGDOM',
 'SAUDI ARABIA',
 'PAKISTAN',
 'BANGLADESH',
 'ITALY',
 'TURKEY',
 'ARGENTINA',
 'GERMANY',
 'FRANCE',
 'IRAQ',
 'PHILIPPINES',
 'INDONESIA',
 'CANADA',
 'QATAR',
 'KAZAKHSTAN',
 'EGYPT',
 'ECUADOR',
 'BOLIVIA',
 'SWEDEN',
 'OMAN',
 'ISRAEL',
 'UKRAINE',
 'DOMINICAN REPUBLIC',
 'PANAMA',
 'BELGIUM',
 'KUWAIT',
 'BELARUS',
 'UAE',
 'ROMANIA',
 'NETHERLANDS',
 'SINGAPORE',
 'GUATEMALA',
 'PORTUGAL',
 'POLAND',
 'NIGERIA',
 'HONDURAS',
 'BAHRAIN',
 'JAPAN',
 'ARMENIA',
 'GHANA',
 'KYRGYZSTAN',
 'AFGHANISTAN',
 'SWITZERLAND',
 'ALGERIA',
 'AZERBAIJAN',
 'MOROCCO',
 'UZBEKISTAN',
 'SERBIA',
 'MOLDOVA',
 'IRELAND',
 'KENYA',
 'VENEZUELA',
 'NEPAL',
 'AUSTRIA',
 'COSTA RICA',
 'ETHIOPIA',
 'AUSTRALIA',
 'EL SALVADOR',
 'CZECHIA',
 'CAMEROON',
 'IVORY COAST',
 'S. KOREA',
 'DENMARK',
 'PALESTINE',
 'BOSNIA AND HERZEGOVINA',
 'BULGARIA',
 'MADAGASCAR',
 'SUDAN

In [75]:
list(set(tloc1)-set(cname))

['EUROPE', 'NORTH AMERICA']

### group location 

In [76]:
UK,CANADA,NETHERLANDS,AUS,REMOVE,INDIA,MEXICO,EUROPE,NA,SPAIN,NIGERIA,BRAZIL,GERMANY,UAE,IRELAND=[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]

In [49]:
UK.extend(('ESSEX','MANCHESTER','SCOTLAND','GLASGOW','WALES','LONDON','OXFORD',
           'UK','NORTHERN IRELAND','LIVERPOOL','ENGLAND','EDINBURGH','BELFAST'))
CANADA.extend(('ONTARIO CANADA','SASKATCHEWAN','MONTREAL','QUÉBEC','VICTORIA','NOVA SCOTIA',
               'VANCOUVER','BRITISH COLUMBIA','MANITOBA','TORONTO','BC','B.C.','🇨🇦','ALBERTA'))
NETHERLANDS.extend(('NEDERLAND','THE NETHERLANDS'))
AUS.extend(('WESTERN AUSTRALIA','QUEENSLAND','MELBOURNE','NEW SOUTH WALES','SYDNEY'))
REMOVE.extend(('HERE','WORLDWIDE', 
 'HELL','HE/HIM','PLANET EARTH','SHE/HER','EVERYWHERE','GLOBAL','THE WORLD','ON','EARTH','WORLD',
 'AB','SOMEWHERE','NIG'))
INDIA.extend(('NEW DELHI','MUMBAI','भारत'))
MEXICO.append('MÉXICO')
EUROPE.append('EUROPE')
NA.extend(('PACIFIC NORTHWEST','PNW'))
SPAIN.append('ESPAÑA')
NIGERIA.extend(('LAGOS','ABUJA'))
BRAZIL.append('BRASIL')
GERMANY.append('DEUTSCHLAND')
UAE.append('UNITED ARAB EMIRATES')
IRELAND.append('DUBLIN')
